In [70]:
import os
import time
import pandas as pd
import requests

# pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup # pip install beautifulsoup4

from datetime import datetime

## Definir URL inical 

Inicialmente vamos a utilizar una URL que **parece** que nos lleva a un buscador generico de **todas** las Teses, dissertações e produtos pós-doutorado ordenados por titulo. 

Aunque inicialmente se pedia extraer la informacion por secciones o campos de investigacion ( Ej : Ciencias politicas), como la estructura de la página es siempre igual, la idea es generar un codigo que extraiga informacion en funcion de la extructura de la pagina y no tanto de el filtro en concreto que se haya aplicado a dicha pagina, por ejemplo al filtrar solo un tipo de publicaciones segun tematica

In [71]:
url = "http://repositorio2.unb.br/jspui/handle/10482/45731/browse?type=title&sort_by=1&order=ASC&rpp=100&etal=-1&null=&"

## Workflow 

La pagina tiene la siguiente extructura (**pagina de busqueda**) 

- Contiene una tabla, en la que cada fila hace referencia a una publicacion.
- En cada fila podemos ver varios campos referentes al articulo, entre ellos, el titlo y el link a la pagina especifica de el articulo concreto (**pagina de articulo**) 
- Se puede condicionar los resultados de la tabla a un numero X de entradas.
- En este caso hemos escogido que nos devuleva el mayor numero de resultados por tabla para cada "pagina" en este caso **100**
- Al principio y al final de la tabla tenemos unos ecabezados donde podemos encontrar botones de "Siguiente" y "Anterior" que nos permiten navegar a las siguientes paginas de busqueda. 

El flujo de extraccion sigue la siguiente logica:

1. Comenzamos en la "pagina de busqueda" inicial (que muestra los articulos del 1 a al 100) **star_function**
2. Localizamos para todas las filas/articulos (100) de la página el elemnto de hhtml que hace referencia al titulo y que contiene el hipervinculo a la pagina del articulo correspondiente.
3. Para cada uno de los articulos / elementos de las filas, accederemos al hipervinculo y entraremos a la url corespondiente al articulo **página de articulo**
    - Extraemos la infomacion del articulo visitado
    - volvemos a la pagina inicial
4. Una vez extraida toda la informacion de los articulos (100) accedemos a la pagina Siguiente y volvemos a ejecutar el punto 2 y 3.
5. Cuando el botçon de siguiente deje de aparecer querra decir que hemos llegado a la ultima pagina de busqueda y que por tanto hemos "escaneado" todos los articulos de referencia. Esto querra decir que hemos finalizado el trabajo. 

### 1. Start Function

Creamos una funcio para inciciar el driver, dada una url. 

Es importante destacar que hemos configurado el dirver para bloquear las descargas, esto es debido a que hemos visto, en la practica que ejecutando el codigo se iniciaban descargas de los documentos (los pdf's de las publicaciones) de forma automatica. 

Para evitar esto, configuramos las opciones para bloquear las descargas automaticas

In [84]:
def open_url_block_dw(url):

    # Configurar ChromeOptions para bloquear descargas
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "download.prompt_for_download": False,   # No mostrar el diálogo de descarga
        "download_restrictions": 3  # Bloquear todas las descargas automáticas
    })

    # Inicializar el controlador de Chrome con las opciones
    driver = webdriver.Chrome(service=Service(), options=chrome_options)
    
    driver.get(url)
    
    return driver


### 2. Funciones de movimiento:

In [85]:
"""

LO SACAMOS AL MAIN, CREO QUE QUEDA MAS CLARO 

def page_movement(driver):
    
    n = 1
    
    header = driver.find_element(By.CLASS_NAME, "panel-heading.text-center").text
    
    while "Siguiente" in header:
        
        print(f"pagina {n}")
        
        # ejecuto target papers
        target_papers(driver)
        
        driver = next_page(driver)
        
        n +=1
        header = driver.find_element(By.CLASS_NAME, "panel-heading.text-center").text
        
    print("Se han escaneado todas las páginas")
    
"""

'\n\nLO SACAMOS AL MAIN, CREO QUE QUEDA MAS CLARO \n\ndef page_movement(driver):\n    \n    n = 1\n    \n    header = driver.find_element(By.CLASS_NAME, "panel-heading.text-center").text\n    \n    while "Siguiente" in header:\n        \n        print(f"pagina {n}")\n        \n        # ejecuto target papers\n        target_papers(driver)\n        \n        driver = next_page(driver)\n        \n        n +=1\n        header = driver.find_element(By.CLASS_NAME, "panel-heading.text-center").text\n        \n    print("Se han escaneado todas las páginas")\n    \n'

In [86]:
def next_page(driver):
    
    footer = driver.find_element(By.CLASS_NAME, "panel-footer.text-center")
    
    footer.find_element(By.CLASS_NAME, "pull-right").click()
    
    return driver
    

### 3. Localizar articulos

In [91]:
def target_papers(driver):
    
    info_list = []
    
    # Find all elements containig title/link to a publiced article:
    elements = driver.find_elements(By.XPATH, '//td[@headers="t3"]')
    
    #loop over papers to get the relevant information:
    for e in elements:
        
        # target the element that contains the link to the paper page:
        href = e.find_element(By.TAG_NAME, "a")
    
        # Go to the paper link page:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(href)).click()
        
        # Extract relevant information return a dictionary to be added to the df
        info = extract_info(driver)
        
        info_list.append(info)

        # Go back to searh page (principal page)
        driver.back()
        
    df = pd.DataFrame(info_list)    
        
    return driver, df

### 4. Extraer informacion

In [98]:
def extract_info(driver):
    
    # Get the html code form the page (article_page)
    page = driver.page_source

    # parse the information to be able to acces and manipulate
    info = BeautifulSoup(page, "html.parser")

    #Create an empty dictionaty to store all values:

    data_dictionay = {}

    # Store the document link:
    try:
        data_dictionay["Documento"] = f"{driver.current_url}/1/{info.find('td', headers='t1').text}"
    except:
        data_dictionay["Documento"] ="No hay ficheros asociados a este ítem"

    """ 
    NOTA IMPORTANTE:
    Que pasa si hay mas que uno?
    PArece que el link se compone de el link del articulo + /1/+ el nombre
    por eso digo lo de que pasa si hay mas de uno 
    """
    

    # Select table, where all the information is contain
    table = info.find("table", class_="table itemDisplayTable")

    # Find all rows inside the table:
    rows = table.find_all("tr")

    # Iterate trought all rows to get the field_name and the field_content.
    # Note: having in mind multipliciy of the field_content

    for r in rows:

        # Find data inside each row (includes filed_name and field_content)
        data = r.find_all("td") # La fila esta compuesto por dos columnas o celdas

        # field_name is contain in the first element of "data"
        field_name = data[0]

        # field_name is contain in the second element of "data"
        field_content = data[1]


        # Extract the text inside both field_name and field_content:

        #field_name_text wold be or key (we add some formating for avoiding problems in the dictionary)
        key = field_name.text.replace(":","").strip()

        #field_content_text wold be or value/s

        "NOTE: There are some fields that can have more than one content, as mentioned before"

        if field_content.find_all("a") != []:

            values = [i.text for i in field_content.find_all("a")]

        else:
            values = field_content.text

        data_dictionay[key] = values


    return data_dictionay

# main():

In [118]:
def main(url, name):
    
    #start driver:
    driver = open_url_block_dw(url)

    #create data frame to store info
    df = pd.DataFrame()

    # track record of activity 
    n = 1

    header = driver.find_element(By.CLASS_NAME, "panel-heading.text-center").text

    while "Siguiente" in header:

        print(f"pagina {n}")

        # ejecuto target papers
        driver, df1 = target_papers(driver)

        df = pd.concat([df, df1], axis=0)

        driver = next_page(driver)

        n +=1
        header = driver.find_element(By.CLASS_NAME, "panel-heading.text-center").text

    # Last Page:
    print(f"pagina {n}")
    driver, df1 = target_papers(driver)
    df = pd.concat([df, df1], axis=0)
    

    print("Se han escaneado todas las páginas")
    
    df.to_csv(f"{name}.csv", index = True)
    
    print(f" Data Frame {name} exportado con exito")
    
    
    driver.quit()
    
    return df

In [119]:
pwd

'/Users/Moni/GitHub-Projects/mosaico-database'

In [120]:
url = "http://repositorio2.unb.br/jspui/handle/10482/45731/browse?type=ppg&order=ASC&rpp=20&value=Programa+de+P%C3%B3s-Gradua%C3%A7%C3%A3o+em+Ci%C3%AAncias+Sociais+-+Estudos+Comparados+sobre+as+Am%C3%A9ricas"

name = "todos_los_docs"

df = main(url, name)

pagina 1
Se han escaneado todas las páginas
 Data Frame PG em Ciências Sociais - Estudos Comparados sobre as Américas exportado con exito
pagina 1
pagina 2
pagina 3


KeyboardInterrupt: 

In [121]:
df

,Documento,Título,Autor,Orientador(es),Assunto,Fecha de publicación,Data de defesa,Citación,Resumen,Abstract,...,Descripción,metadata.dc.description.ppg,Licença,Aparece en las colecciones,Agência financiadora,Otros títulos,Coorientador(es),DOI,Résumé,metadata.dc.contributor.email
0,http://repositorio2.unb.br/jspui/handle/10482/...,O acesso à justiça para as mulheres indígenas ...,"[Ferro, Larissa Cristina de Sousa]","[Igreja, Rebecca Forattini Altino Machado Lemos]","[Bolívia, Mulheres indígenas, Acesso à justiça...",9-sep-2019,5-feb-2019,"FERRO, Larissa Cristina de Sousa. O acesso à j...",La presente disertación trata del tema del acc...,This dissertation deals with the issue of acce...,...,Dissertação (mestrado)—Universidade de Brasíli...,[Programa de Pós-Graduação em Ciências Sociais...,A concessão da licença deste item refere-se ao...,"[Teses, dissertações e produtos pós-doutorado]",NaN,NaN,NaN,NaN,NaN,NaN
1,http://repositorio2.unb.br/jspui/handle/10482/...,Agendas políticas de movimentos de indígenas m...,"[Ferro, Larissa Cristina de Sousa]","[Ruano Ibarra, Elizabeth]","[Povos indígenas, Movimento indígena - Bolívia...",8-ene-2024,19-may-2023,"FERRO, Larissa Cristina de Sousa. Agendas polí...",Analizo em perspectiva comparada las agendas p...,I analyze the political agendas of indigenous ...,...,"Tese (doutorado) — Universidade de Brasília, I...",[Programa de Pós-Graduação em Ciências Sociais...,NaN,"[Teses, dissertações e produtos pós-doutorado]",NaN,NaN,NaN,NaN,NaN,NaN
2,http://repositorio2.unb.br/jspui/handle/10482/...,O alinhamento geopolítico do Brasil e seus imp...,"[Pereira Filho, Elcy Gomes]","[Silva, Leonardo Cavalcanti da]","[Migrações internacionais, Geopolítica - Brasi...",1-feb-2024,27-sep-2023,"PEREIRA FILHO, Elcy Gomes. O alinhamento geopo...",O presente estudo tem por objetivo analisar o ...,The present study aims to analyse the geopolit...,...,Dissertação (mestrado)—Universidade de Brasíli...,[Programa de Pós-Graduação em Ciências Sociais...,A concessão da licença deste item refere-se ao...,"[Teses, dissertações e produtos pós-doutorado]",NaN,NaN,NaN,NaN,NaN,NaN
3,http://repositorio2.unb.br/jspui/handle/10482/...,Autonomia indígena no estado plurinacional bol...,"[Pereira, Pedro Franco de Carvalho da Silva]","[Moreira, Elaine]","[Indígenas, Bolívia - política e governo]",3-jul-2024,7-nov-2023,"PEREIRA, Pedro Franco de Carvalho da Silva. Au...",El conjunto de organizaciones indígenas latino...,NaN,...,Dissertação (mestrado)—Universidade de Brasíli...,[Programa de Pós-Graduação em Ciências Sociais...,A concessão da licença deste item refere-se ao...,"[Teses, dissertações e produtos pós-doutorado]",Coordenação de Aperfeiçoamento de Pessoal de N...,NaN,NaN,NaN,NaN,NaN
4,http://repositorio2.unb.br/jspui/handle/10482/...,"Autonomia, geopolítica crítica e colonialidade...","[Silva, Carolina Albuquerque]","[Menezes, Roberto Goulart]","[Integração regional - América Latina, Autonom...",1-feb-2024,31-mar-2023,"SILVA, Carolina Albuquerque. Autonomia, geopol...",Existe una tradición de reflexión en torno a l...,There is an academic tradition around the idea...,...,"Tese (doutorado)—Universidade de Brasília, Ins...",[Programa de Pós-Graduação em Ciências Sociais...,A concessão da licença deste item refere-se ao...,"[Teses, dissertações e produtos pós-doutorado]",NaN,NaN,NaN,NaN,NaN,NaN
5,http://repositorio2.unb.br/jspui/handle/10482/...,A comunicação alternativa em Oaxaca/México : r...,"[Tavares, Clarissa Noronha Melo]","[Silva, Cristhian Teófilo da]","[Comunicação, Indígenas - línguas]",30-mar-2011,20-may-2010,"TAVARES, Clarissa Noronha Melo. A comunicação ...",A presente pesquisa tem como tema central de i...,NaN,...,Dissertação (mestrado) -- Universidade de Bra...,[Programa de Pós-Graduação em Ciências Sociais...,NaN,"[Teses, dissertações e produtos pós-doutorado]",NaN,NaN,NaN,NaN,NaN,NaN
6,http://repositorio2.unb.br/jspui/handle/10482/...,"Cuando el cuy tuvo que salir de la cocina, int...","[Perche, Guillaume Maur

In [ ]:
PArece que esta universidad es muy similar. Podria creo que aplicar casi el mismo codigo o muy parecido:
    
    Me da la sensacion de que si el value lo especificas solo filtra uno pero sino puedes acceder directa,ente a todos los repositorios y categorias
    
    Estaria bien ver todas las universidades que categorias tienen por si hay que unificar o crear equivalencias 


"https://tedebc.ufma.br/jspui/browse?type=program&order=DESC&rpp=20&value="


Parece que todas tienen una estructura similar, a ver que puedo hacer para ajustar el codigo y que funcione para todas o al go parecido 
molaria tenr variables en funcion de la uni y que se configura el codigo para eso 


PD: Esto esta muy guay pero tienes que hacer tus dos aplicaciones diarias, guapa, que sino no vas a tener curro 